1. Remover colunas com dados faltando (a menos que maior parte dos dados na coluna estejam faltando, não é uma boa opção)
2. Imputation: preencher os valores faltando com algum valor (média, valor mais próximo, ...)
3. Extendendo Imputation: pode adicionar coluna para informar que foi inputado um dado naquela opisção

In [1]:
# código já pronto
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("melb_data.csv")

y = data["Price"]

# previsão numérica
melb_predictors = data.drop(["Price"], axis=1)
X = melb_predictors.select_dtypes(exclude=["object"])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# funão para comparar diferentes métodos
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    
    return mean_absolute_error(y_valid, preds)

### Score com abordagem 1 (dropar as colunas com valores faltanmdo)

In [6]:
# nome das colunas que tem dados faltando
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# dropando as colunas
reduced_X_train = X_train.drop(cols_with_missing, axis=1)  # axis=1 == coluna
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE - 1: ")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE - 1: 
183550.22137772635


### Score com abordagem 2 (Imputation)
SimpleImputer para trocar valores faltando pela média em cada coluna

In [17]:
from sklearn.impute import SimpleImputer

# imputando
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# imputation tira nome das colunas; inserindo eles de novo
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE - 2:")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE - 2:
178166.46269899711


### Score com abordagem 3 (Extensão de Imputation)


In [12]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# nova coluna indiciando que vai ser inputado
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [16]:
# shape do treinamento
print(X_train.shape)

# numero de valores faltando em cada coluna nos dados de treinamento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64
